In [1]:
import torch
from dataset import get_imagenet256_dataset
import matplotlib.pyplot as plt
import torchvision
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pathlib
from tqdm import tqdm
import os
from torchvision.transforms.functional import to_pil_image
from concurrent.futures import ThreadPoolExecutor

In [2]:
os.cpu_count()

128

In [3]:
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

ood_dataset = get_imagenet256_dataset(datadir='./datasets')
# https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split
batch_size = 100
loader = torch.utils.data.DataLoader(ood_dataset, batch_size=batch_size,
                                        shuffle=True, generator=torch.Generator().manual_seed(0), num_workers=8)

savedir = './datasets/imagenet50K'
max_iters = 50000 // batch_size
def save_image(image, idx):
    to_pil_image(image).save(os.path.join(savedir, f'{idx:06d}.png'))
pathlib.Path(savedir).mkdir(parents=True, exist_ok=True)
for i, data in tqdm(zip(range(max_iters), loader), total=max_iters):
    seed_imgs = data[0] if isinstance(data, list) else data
    with ThreadPoolExecutor(max_workers=20) as executor:
        for j in range(seed_imgs.shape[0]):
            executor.submit(save_image, seed_imgs[j], i * batch_size + j)


100%|██████████| 500/500 [03:03<00:00,  2.72it/s]
